# Imports

In [ ]:
import holoviews as hv
import hvplot.streamz
import numpy as np
import pandas as pd
import panel as pn
from streamz.dataframe import PeriodicDataFrame

IDX = pd.IndexSlice

In [ ]:
hv.extension("bokeh")

# Streaming

In [ ]:
state = {}
rows_per_timepoint = 10


def new_df():
    return pd.DataFrame(
        {
            "RFP-Penta/intensity_mean": np.random.uniform(0, 5000, rows_per_timepoint),
            "YFP-DUAL/intensity_mean": np.random.uniform(0, 5000, rows_per_timepoint),
            "area": np.random.normal(150, 50, rows_per_timepoint),
            "axis_major_length": np.random.normal(20, 5, rows_per_timepoint),
            "axis_minor_length": np.random.normal(10, 5, rows_per_timepoint),
        }
    )


def poll_table(last, now, **kwargs):
    counter = state.setdefault("counter", 0) + 1
    state["counter"] = counter
    if "df" not in state:
        state["df"] = new_df()
    else:
        state["df"] = pd.concat((state["df"], new_df()), axis=0, copy=True)
    return state["df"]


# if I set this to 100ms, it completely breaks for me
measurements_stream = PeriodicDataFrame(poll_table, interval="300ms")

In [ ]:
def filter_plots(data, singles, pairs):
    # return pn.Column(pn.pane.HoloViews(hv.Layout([data.hvplot.kde("area", yaxis="bare"), data.hvplot.kde("axis_major_length", yaxis="bare")])))
    # sel = link_selections.instance()
    return pn.Column(
        pn.pane.HoloViews(
            # link_selections(
            hv.Layout(
                [
                    # hv.Distribution(data, k).opts(height=200, width=200, yaxis="bare")
                    # data.hvplot.kde(k, height=200, responsive=True, yaxis="bare")
                    data.hvplot.kde(
                        k, height=200, width=200, yaxis="bare", backlog=1000
                    )
                    for idx, k in enumerate(singles)
                ]
            ).cols(6),
            # sizing_mode="stretch_width",
        ),
        pn.pane.HoloViews(
            # link_selections(
            hv.Layout(
                [
                    data.hvplot.scatter(
                        *k,
                        height=300,
                        width=300,
                        hover=False,
                        size=2,
                        backlog=1000,
                    )
                    for k in pairs
                ]
            ),
            # ),
            sizing_mode="stretch_width",
        ),
    )


p = filter_plots(
    measurements_stream,
    [
        "RFP-Penta/intensity_mean",
        "YFP-DUAL/intensity_mean",
        "area",
        "axis_minor_length",
        "axis_major_length",
    ],
    [
        ("RFP-Penta/intensity_mean", "YFP-DUAL/intensity_mean"),
        ("axis_minor_length", "axis_major_length"),
        ("area", "RFP-Penta/intensity_mean"),
    ],
)
p